In [1]:
!pip install --upgrade google-cloud-aiplatform google-cloud-logging --quiet
!pip install "google-cloud-aiplatform[evaluation]" --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 42.7 MB/s eta 0:00:00


#### ==> Restart the session now

In [1]:
import pandas as pd
import logging
import google.cloud.logging
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PointwiseMetric,
)

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

pd.set_option("display.max_colwidth", None)

In [2]:
PROJECT_ID = "qwiklabs-gcp-01-8522da238c4c"
LOCATION = "us-central1"

import vertexai

# Initialize vertexai
# vertexai.init()
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Do not remove logging section
log_message = f"Vertex AI initialize: {vertexai}"
logging.info(log_message)

INFO:root:Vertex AI initialize: <module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>


## Task 2. Explore example data and make a simple summarization request to a Gen AI model

In [3]:
prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}"

In [4]:
system_prompt=["You are an retail domestic merchandise expert"]

question=["Provide a one sentence summary of the following text"]

description=[
  "Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated",
  "Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.",
  "Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats.",
  "OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece.",
  "Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer cycles. Plus, with smart connectivity, you can monitor and control your wash right from your phone — because clean dishes should never slow you down. "
]


In [5]:
flash_model = GenerativeModel(
  "gemini-2.0-flash",
  generation_config={
      "temperature": 0,
  },
)

In [6]:
llm_response = flash_model.generate_content(
  # Fill in the appropriate configuration
  prompt_template.format(system_prompt=system_prompt, question=question, description=str(description[1]))
)

Markdown(llm_response.text)

# Do not remove logging section
log_message = f"Markdown output: {llm_response.text}"
logging.info(log_message)

INFO:root:Markdown output: This floral midi dress is a lightweight and breathable option for summer, featuring a flattering silhouette with a cinched waist, A-line skirt, and delicate details like spaghetti straps and ruffle accents.



## Task 3. Prepare the Evaluation Dataset, EvalTask and evaluate of a single model

In [13]:
system_prompt_data = [system_prompt] * 5
question_data = [question] * 5

# Create a Pandas DataFrame for your evaluation dataset
dataset = pd.DataFrame({
    "system_prompt": system_prompt_data,
    "question": question_data,
    "description": description
})

In [16]:
dataset[0:1]

,system_prompt,question,description
0,[You are an retail domestic merchandise expert],[Provide a one sentence summary of the following text],"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated"


In [17]:
pointwise_result = EvalTask(
    dataset=dataset,
    metrics=[MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY],
    experiment='product-summarization-quality',
).evaluate(
    model=flash_model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    experiment_run_name="gemini-summarization-quality-eval-run",
)

INFO:root:MetadataStore default not found.


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash', 'temperature': 0}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash.
100%|██████████| 5/5 [00:00<00:00,  7.48it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.6940786170000592 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 5 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 5/5 [00:02<00:00,  1.68it/s]
INFO:vertex

In [18]:
pointwise_result.summary_metrics

{'row_count': 5,
 'summarization_quality/mean': np.float64(5.0),
 'summarization_quality/std': 0.0}

In [19]:
pointwise_result.metrics_table

,system_prompt,question,description,prompt,response,summarization_quality/explanation,summarization_quality/score
0,[You are an retail domestic merchandise expert],[Provide a one sentence summary of the following text],"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","These tailored men's blue dress shorts offer a refined and comfortable option for warm-weather, smart-casual occasions.\n","The response followed the instructions to provide a one sentence summary of the provided text, the response is grounded, concise, and fluent.",5.0
1,[You are an retail domestic merchandise expert],[Provide a one sentence summary of the following text],"Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","This floral midi dress is a lightweight and breathable option for summer, featuring a flattering silhouette with a cinched waist, A-line skirt, and delicate details like spaghetti straps and ruffle accents.\n","The response follows the instructions by providing a one sentence summary of the text, is grounded in the provided text, is concise, and fluent.",5.0
2,[You are an retail domestic merchandise expert],[Provide a one sentence summar

## Task 4. Evaluate and compare 2 models

In [20]:
baseline_model = GenerativeModel("gemini-2.0-flash-lite")
candidate_model = GenerativeModel("gemini-2.0-flash")

PAIRWISE_METRIC = PairwiseMetric(
    metric="pairwise_summarization_quality",
    metric_prompt_template=MetricPromptTemplateExamples.get_prompt_template(
        "pairwise_summarization_quality"
    ),
    baseline_model=baseline_model,
)

In [21]:
pairwise_result = EvalTask(
    dataset=dataset,
    metrics=[PAIRWISE_METRIC],
    experiment="pairwise-product-summarization-quality",  # (!)change this(!)
).evaluate(
    model=candidate_model,
    prompt_template="# System_prompt\n{system_prompt} # Question\n{question} # Description {description}",
    experiment_run_name="gemini-summarization-quality-eval-run",
)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '# System_prompt\n{system_prompt} # Question\n{question} # Description {description}', 'model_name': 'publishers/google/models/gemini-2.0-flash'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash.
100%|██████████| 5/5 [00:00<00:00,  9.52it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 0.5327162490000319 seconds.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-2.0-flash-lite.
100%|██████████| 5/5 [00:00<00:00,  6.54it/s]
INFO:vertexai.evaluation._evaluation:

In [22]:
pairwise_result.summary_metrics

{'row_count': 5,
 'pairwise_summarization_quality/candidate_model_win_rate': np.float64(0.0),
 'pairwise_summarization_quality/baseline_model_win_rate': np.float64(0.6)}

In [23]:
pairwise_result.metrics_table

,system_prompt,question,description,prompt,response,baseline_model_response,pairwise_summarization_quality/explanation,pairwise_summarization_quality/pairwise_choice
0,[You are an retail domestic merchandise expert],[Provide a one sentence summary of the following text],"Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated","These men's blue dress shorts offer a refined, comfortable, and versatile option for warm-weather, smart-casual occasions.\n","These men's blue dress shorts offer a stylish and comfortable option for warm-weather wear, featuring a slim fit and cotton-blend fabric suitable for smart-casual occasions.\n","Both responses follow instructions, are grounded in the context, and are concise and fluent.",TIE
1,[You are an retail domestic merchandise expert],[Provide a one sentence summary of the following text],"Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","# System_prompt\n['You are an retail domestic merchandise expert'] # Question\n['Provide a one sentence summary of the following text'] # Description Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.","This lightweight floral midi dress features a flattering silhouette with a cinched waist and A-line skirt, perfect for warm weather occasions.\n","This summer floral midi dress offers a lightweight, elegant design with a flattering silhouette and vibrant fl